# Ensembl numeric data extraction (positive)

## Below function to get Ensembl IDs from .csv and convert to a python list in JSON format

In [1]:
def csv_to_id(path):
    df = pd.read_csv(path)
    ids = df.TEST_pos.tolist()
    
    for loc in ids:
        loc = str(loc)   #here 'nan' is converted to a string to compare with if
        if loc != 'nan': 
            cleaned_ids.append(loc)
    cleaned = json.dumps(cleaned_ids)        
    correct_format = "{" +'"ids": ' + cleaned + "}"
    return correct_format

In [2]:
import pandas as pd
import numpy as np
import json
cleaned_ids = []
path = '/Training set.example.csv'

cleaned_IDs = csv_to_id(path)


In [99]:
#print(cleaned_IDs)

## Passing the list to Ensembl REST API to get JSON response 

In [2]:
# Single request, multiple IDs
import requests, sys
import json, urllib
server = "https://rest.ensembl.org"
ext = '/lookup/id/?format=full;expand=1;utr=1;phenotypes=1'
#ext = '/lookup/id/?
headers = {'Content-Type' : 'application/json', "Accept" : 'application/json'}
#'{"ids" : ["ENSG00000255689", "ENSG00000254443"]}'
#cleaned_IDs = {"ids": ["ENSG00000255689", "ENSG00000254443"]}
r = requests.post(server+ext,headers=headers, data='{0}'.format(cleaned_IDs))

print(str(r))
print(type(r))

decoded = r.json()
#print(repr(decoded))

## Saving JSON response on local machine and then loading the .json file

In [15]:
import json
with open('/positive_data.json', 'w') as outfile:
    json.dump(decoded, outfile, indent=4, sort_keys=True)    

In [3]:
with open('/positive_data.json') as access_json:
    read_content = json.load(access_json)

In [5]:
if read_content[cleaned_ids[145]] == None: #Null gene ID present
    print(True)

## 'read_content' variable contains the json response received

In [6]:
gene_display_name = []
gene_start = []
gene_end = []
gene_strand = []
gene_seq_region_name = []
null_gene_ids = []
gene_ids = []
gene_biotype = []

## Below function [get_gene_data() ] to extract 'gene' data. Data Extracted are :
1. gene display_name
2. gene start
3. gene end
4. gene strand
5. gene seq_region_name
6. gene biotype

In [7]:
def get_gene_data():
    
    count = 0
    for i in range(len(cleaned_ids)):
        if read_content[cleaned_ids[i]] == None:
            null_gene_ids.append([cleaned_ids[i]])
            #del cleaned_ids[i]
        else:
            
            gene_ids.append(cleaned_ids[i])
            gene_display_name.append(read_content[cleaned_ids[i]]['display_name'])
            gene_start.append(read_content[cleaned_ids[i]]['start'])
            gene_end.append(read_content[cleaned_ids[i]]['end'])
            gene_strand.append(read_content[cleaned_ids[i]]['strand'])
            gene_seq_region_name.append(read_content[cleaned_ids[i]]['seq_region_name'])
            gene_biotype.append(read_content[cleaned_ids[i]]['biotype'])
            if cleaned_ids[i] in read_content:
                count = count + 1
        
    print(count) 

In [8]:
get_gene_data()

## 'null_gene_ids' variable contains the IDs having null values

In [9]:
print('No. of Null IDs are {0}'.format(len(null_gene_ids)))
print('Null IDs are :')
for i in range(len(null_gene_ids)):
    
    print(null_gene_ids[i])

In [10]:
print('No. of contents of gene_start is {0}'.format(len(gene_start)))
print('No. of contents of gene_end is {0}'.format(len(gene_end)))
print('No. of contents of gene_strand is {0}'.format(len(gene_strand)))
print('No. of contents of gene_seq_region_name is {0}'.format(len(gene_seq_region_name)))
print('No. of contents of gene_display_name is {0}'.format(len(gene_display_name)))

In [11]:
cleaned_ids = []
cleaned_ids = gene_ids
print(len(cleaned_ids))

In [12]:
no_of_transcripts = []
gene_ids_for_transcripts = []

## Below function [ get_no_of_transcripts() ] to calculate no. of transcripts in a particular gene

In [13]:
def get_no_of_transcripts():
    for i in range(len(cleaned_ids)):
        no_of_transcripts.append(len(read_content[cleaned_ids[i]]['Transcript']))
        
        for k in range(len(read_content[cleaned_ids[i]]['Transcript'])):
            gene_ids_for_transcripts.append(cleaned_ids[i])
            
    for j in range(len(cleaned_ids)):
        print('No. of transcripts in gene "{0}" are {1}'.format(cleaned_ids[j],no_of_transcripts[j]))

In [14]:
get_no_of_transcripts()

In [111]:
#read_content[cleaned_ids[0]]['Transcript'][0]

In [15]:
transcript_id = []
transcript_start = []
transcript_end = []
transcript_biotype = []

In [16]:
#gene_ids_for_transcripts

## Below function [get_transcript_data() ] to extract 'transcript' data. Data Extracted are :
1. transcript id
2. transcript start
3. transcript end
4. transcript biotype

In [17]:
def get_transcript_data():
    for i in range(len(cleaned_ids)):
        
        for j in range(len(read_content[cleaned_ids[i]]['Transcript'])):
            transcript_id.append(read_content[cleaned_ids[i]]['Transcript'][j]['id'])
            transcript_start.append(read_content[cleaned_ids[i]]['Transcript'][j]['start'])
            transcript_end.append(read_content[cleaned_ids[i]]['Transcript'][j]['end'])
            transcript_biotype.append(read_content[cleaned_ids[i]]['Transcript'][j]['biotype'])
            
    for k in range(len(gene_ids_for_transcripts)):
        print('Transcript "{0}" of gene ID "{1}" has start and end as : "{2}" & "{3}"'.format(transcript_id[k],gene_ids_for_transcripts[k],transcript_start[k],transcript_end[k]))

In [18]:
get_transcript_data()

In [ ]:
print(len(transcript_id))
print(len(transcript_start))
print(len(transcript_end))
print(len(gene_ids_for_transcripts))

In [ ]:
len(read_content[cleaned_ids[0]]['Transcript'][0]["Exon"])

In [19]:
no_of_exons = []
transcript_ids_for_exons = []

## Below function [ get_no_of_exons() ] to calculate no. of exons for a particular transcript

In [20]:
def get_no_of_exons():
    for i in range(len(cleaned_ids)):
        for j in range(len(read_content[cleaned_ids[i]]['Transcript'])):
            no_of_exons.append(len(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"]))
        
            for k in range(len(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"])):
                transcript_ids_for_exons.append(transcript_id[j])
            
    for l in range(len(cleaned_ids)):
        print('No. of exons in transcript "{0}" are {1}'.format(transcript_id[l],no_of_exons[l]))

In [ ]:
len(read_content[cleaned_ids[0]]['Transcript'][0]["Exon"])

In [21]:
get_no_of_exons()

In [ ]:
sum(no_of_exons)

In [ ]:
len(transcript_ids_for_exons)

In [ ]:
read_content[cleaned_ids[0]]['Transcript'][0]["Exon"][0]

In [22]:
exon_id = []
exon_start = []
exon_end = []
gene_ids_for_exons = []

## Below function [get_exon_data() ] to extract 'exon' data. Data Extracted are :
1. exon id
2. exon start
3. exon end

In [23]:
def get_exon_data():
    for i in range(len(cleaned_ids)):
        
        for j in range(len(read_content[cleaned_ids[i]]['Transcript'])):
            for k in range(len(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"])):
                
                
                exon_id.append(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['id'])
                exon_start.append(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['start'])
                exon_end.append(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['end'])
                gene_ids_for_exons.append(cleaned_ids[i])
        
            
    for l in range(len(transcript_ids_for_exons)):
        
        print('Exon "{0}" of Transcript ID "{1}" having gene ID "{2}" has start and end as : "{3}" & "{4}"'.format(exon_id[l],transcript_ids_for_exons[l],gene_ids_for_exons[l],exon_start[l],exon_end[l]))
    

In [24]:
get_exon_data()

In [ ]:
len(exon_id)

In [ ]:
len(gene_ids_for_exons)

In [25]:
transcript_len = []

## Below function[ get_transcript_length() ] to calculate length of transcript

In [26]:
def get_transcript_length():
 #   for i in range(transcript_id):
  #      for j in range(exon)

    for i in range(len(cleaned_ids)):
        
        for j in range(len(read_content[cleaned_ids[i]]['Transcript'])):
            trans_len = 0
            start = 0
            end = 0
            total_exon_len = 0
            for k in range(len(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"])):
                start = read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['start']
                end = read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['end']
                total_exon_len = total_exon_len + (end - start + 1)

            transcript_len.append(total_exon_len)
    
    for k in range(len(transcript_id)):
        print('Transcript ID "{0}" has length of {1} bps'.format(transcript_id[k], transcript_len[k]))
                
        

In [ ]:
len(transcript_id)

In [27]:
get_transcript_length()

In [ ]:
len(transcript_len)

In [ ]:
transcript_len[-1]

In [ ]:
transcript_id[-1]

In [28]:
exon_len = []

## Below function[ get_exon_length() ] to calculate length of exon

In [29]:
def get_exon_length():
 #   for i in range(transcript_id):
  #      for j in range(exon)
#exon_id
    for i in range(len(cleaned_ids)):
        
        for j in range(len(read_content[cleaned_ids[i]]['Transcript'])):
           # exon_len = 0
           # start = 0
           # end = 0
           # exon_len = 0
            for k in range(len(read_content[cleaned_ids[i]]['Transcript'][j]["Exon"])):
                start = 0
                end = 0
                exon_len_sum = 0
                start = read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['start']
                end = read_content[cleaned_ids[i]]['Transcript'][j]["Exon"][k]['end']
                exon_len_sum = (end - start + 1)

                exon_len.append(exon_len_sum)
    
    for k in range(len(exon_id)):
        print('Exon ID "{0}" has length of {1} bps'.format(exon_id[k], exon_len[k]))
                
        

In [30]:
get_exon_length()

In [ ]:
len(exon_len)

In [ ]:
len(exon_id)

## Exporting gene data to gene_data.csv file

In [31]:
import csv 
header = ['SNO', 'Gene ID', 'Display Name', 'Biotype', 'Start', 'End', 'Strand', 'Seq region Name', 'No. of Transcripts']

path = '/positive_data/gene_data.csv'

with open(path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)


In [32]:
s_no = []
for i in range(len(cleaned_ids)):
    s_no.append(i+1)

In [33]:
import pandas as pd

df = pd.read_csv(path)

df[df.columns[0]] = s_no
df[df.columns[1]] = cleaned_ids
df[df.columns[2]] = gene_display_name
df[df.columns[3]] = gene_biotype
df[df.columns[4]] = gene_start
df[df.columns[5]] = gene_end
df[df.columns[6]] = gene_strand
df[df.columns[7]] = gene_seq_region_name
df[df.columns[8]] = no_of_transcripts


df.to_csv(path)

## Exporting transcript data to transcript_data.csv file

In [34]:
import csv 
header = ['SNO', 'Gene ID', 'Transcript ID', 'Biotype', 'Transcript Start', 'Transcript End', 'Transcript Length','No. of Exons']

path = '/positive_data/transcript_data.csv'

with open(path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)


In [35]:
s_no = []
for i in range(len(transcript_id)):
    s_no.append(i+1)

In [36]:
import pandas as pd

df = pd.read_csv(path)

df[df.columns[0]] = s_no
df[df.columns[1]] = gene_ids_for_transcripts
df[df.columns[2]] = transcript_id
df[df.columns[3]] = transcript_biotype
df[df.columns[4]] = transcript_start
df[df.columns[5]] = transcript_end
df[df.columns[6]] = transcript_len
df[df.columns[7]] = no_of_exons



df.to_csv(path)

## Exporting exon data to exon_data.csv file

In [40]:
import csv 
header = ['SNO', 'Gene ID', 'Transcript ID', 'Exon ID', 'Exon Start', 'Exon End', 'Exon Length']

path = '/positive_data/exon_data.csv'

with open(path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)


In [41]:
s_no = []
for i in range(len(exon_id)):
    s_no.append(i+1)

In [42]:
import pandas as pd

df = pd.read_csv(path)

df[df.columns[0]] = s_no
df[df.columns[1]] = gene_ids_for_exons
df[df.columns[2]] = transcript_ids_for_exons
df[df.columns[3]] = exon_id
df[df.columns[4]] = exon_start
df[df.columns[5]] = exon_end
df[df.columns[6]] = exon_len



df.to_csv(path)